In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import keras

from keras.models import load_model

from keras.layers import Dense, Input, BatchNormalization, LeakyReLU
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.callbacks import EarlyStopping

Dela datan i X och y, träning och validering

In [27]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
y_test[:5]

C:\Users\olofs\AppData\Local\Temp\ipykernel_3704\1754177261.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test[:5]


287    219.0
211     70.0
72     202.0
321    230.0
73     111.0
Name: target, dtype: float64

In [29]:
# find the no of features
n_cols = X_train.shape[1]
n_cols

10

Bygg modell 1

1: instansiera modellen
2: Skapa input lager. Kan vara dela av första dolda lagret, i det här fallet egen rad för tydlighet
3: Lägg till dolda lager. Antal nauroner, aktiveringsfunktion
4: Output-lagret har samma antal neuroner som möjliga utfall. I regression, ett lager. (binär klassificerar har en som är 0-1, multiklass har lika många som antalet klasser)
5: Tänk på att aktiveringsfunktionen i outputlagret måste väljas med hänsyn till typen av problem.

In [30]:
model_1 = Sequential()

model_1.add(Input(shape=n_cols))
model_1.add(Dense(100, activation='relu'))
model_1.add(Dense(100, activation='relu'))
model_1.add(Dense(100, activation='relu'))
model_1.add(Dense(1, activation='relu'))


Modellen måste kompileras.
Optimeringsfunktionen kan väljas om man kan vad de gör.
Loss är den funktion vars värde ska minskas, så mse funkar bra i en regression.
Andra parametrar: metrics

In [31]:
model_1.compile(optimizer='adam', loss='mean_squared_error')


Träna modellen
Validation split indikerar hur mycket av träningsdatan som används i valideringssteget
Epoker är antalet gånger hela datan gås igenom.
Andra parametrar: batch size, callbacks, validation data, 

In [32]:
model_1.fit(X_train, y_train, validation_split = 0.2, epochs = 10)

Epoch 1/10


c:\Users\olofs\anaconda3\envs\machine_learning\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


9/9 [==============================] - 1s 23ms/step - loss: 31500.1133 - val_loss: 22429.4824
Epoch 2/10
9/9 [==============================] - 0s 4ms/step - loss: 31393.3574 - val_loss: 22296.0273
Epoch 3/10
9/9 [==============================] - 0s 4ms/step - loss: 31150.9961 - val_loss: 21979.8242
Epoch 4/10
9/9 [==============================] - 0s 4ms/step - loss: 30570.5098 - val_loss: 21267.8828
Epoch 5/10
9/9 [==============================] - 0s 4ms/step - loss: 29309.4785 - val_loss: 19815.1816
Epoch 6/10
9/9 [==============================] - 0s 5ms/step - loss: 26876.9414 - val_loss: 17195.5371
Epoch 7/10
9/9 [==============================] - 0s 4ms/step - loss: 22708.1113 - val_loss: 13094.7852
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 16621.8418 - val_loss: 7996.6040
Epoch 9/10
9/9 [==============================] - 0s 4ms/step - loss: 9699.7998 - val_loss: 4077.1436
Epoch 10/10
9/9 [==============================] - 0s 4ms/step - loss: 5040.2

In [33]:
y_pred_1 = model_1.predict(X_test)

3/3 [==============================] - 0s 1ms/step


In [34]:
type(y_test)

pandas.core.series.Series

In [35]:
type(y_pred_1)

numpy.ndarray

In [36]:
res_model_1 = mean_squared_error(y_test, y_pred_1)

Build another model

In [37]:
model_2 = Sequential()

model_2.add(Input(n_cols))
model_2.add(Dense(200, activation=keras.activations.tanh))
model_2.add(Dense(100, activation=keras.activations.tanh))
model_2.add(Dense(50, activation=keras.activations.tanh))
model_2.add(Dense(1, activation=keras.activations.swish))

model_2.compile(optimizer='rmsprop', loss=keras.losses.mean_squared_error)

Lägg till callbacks, i det här fallet en earlyt stop.

In [38]:
early_stop = EarlyStopping(patience=5)

callbacks is a list, since you can have sevaral callbacks at the same time

In [39]:
model_2.fit(X_train, y_train, validation_split = 0.2, epochs = 20, callbacks=[early_stop])

Epoch 1/20


c:\Users\olofs\anaconda3\envs\machine_learning\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


9/9 [==============================] - 1s 20ms/step - loss: 31242.1445 - val_loss: 21642.1445
Epoch 2/20
9/9 [==============================] - 0s 5ms/step - loss: 29742.3164 - val_loss: 20521.3848
Epoch 3/20
9/9 [==============================] - 0s 4ms/step - loss: 28855.2461 - val_loss: 20067.7402
Epoch 4/20
9/9 [==============================] - 0s 5ms/step - loss: 28489.7617 - val_loss: 19840.0449
Epoch 5/20
9/9 [==============================] - 0s 6ms/step - loss: 28275.3398 - val_loss: 19683.2031
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 28109.3574 - val_loss: 19552.1895
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 27960.0918 - val_loss: 19429.6934
Epoch 8/20
9/9 [==============================] - 0s 5ms/step - loss: 27818.5605 - val_loss: 19314.1582
Epoch 9/20
9/9 [==============================] - 0s 4ms/step - loss: 27681.1777 - val_loss: 19199.8691
Epoch 10/20
9/9 [==============================] - 0s 4ms/step - loss: 275

In [40]:
y_pred_2 = model_2.predict(X_test)

3/3 [==============================] - 0s 2ms/step


In [41]:
res_model_2 = mean_squared_error(y_test, y_pred_2)

In [42]:
print(f"Result of model_1: {res_model_1} \nResult of model_2: {res_model_2}")

Result of model_1: 3841.9144964185584 
Result of model_2: 21644.697656617824


Build model 3, with the structure of model 2, but the funtions from model 1

In [43]:
model_3 = Sequential()

model_3.add(Input(n_cols))
model_3.add(Dense(200, activation=keras.activations.relu))
model_3.add(Dense(100, activation=keras.activations.relu))
model_3.add(Dense(50, activation=keras.activations.relu))
model_3.add(Dense(1, activation=keras.activations.swish))

model_3.compile(optimizer='rmsprop', loss=keras.losses.mean_squared_error)

early_stop = EarlyStopping(patience=5)
model_3.fit(X_train, y_train, validation_split = 0.2, epochs = 20, callbacks=[early_stop])
y_pred_3 = model_3.predict(X_test)
res_model_3 = mean_squared_error(y_test, y_pred_3)

Epoch 1/20


c:\Users\olofs\anaconda3\envs\machine_learning\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


9/9 [==============================] - 1s 21ms/step - loss: 31470.1211 - val_loss: 22299.2148
Epoch 2/20
9/9 [==============================] - 0s 4ms/step - loss: 30850.2871 - val_loss: 21380.3730
Epoch 3/20
9/9 [==============================] - 0s 5ms/step - loss: 29371.0273 - val_loss: 19872.0723
Epoch 4/20
9/9 [==============================] - 0s 5ms/step - loss: 27080.5859 - val_loss: 17686.9727
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 23840.2734 - val_loss: 14812.2871
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 19705.7988 - val_loss: 11436.4258
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 15209.8613 - val_loss: 8245.0967
Epoch 8/20
9/9 [==============================] - 0s 4ms/step - loss: 10810.2559 - val_loss: 5469.6865
Epoch 9/20
9/9 [==============================] - 0s 9ms/step - loss: 7207.3672 - val_loss: 3892.8284
Epoch 10/20
9/9 [==============================] - 0s 8ms/step - loss: 5026.22

In [44]:
print(f"Result of model_1: {res_model_1} \nResult of model_2: {res_model_2} \nResults of model_3: {res_model_3}")

Result of model_1: 3841.9144964185584 
Result of model_2: 21644.697656617824 
Results of model_3: 3299.3221655739


In [45]:
y_test[0:5]

C:\Users\olofs\AppData\Local\Temp\ipykernel_3704\3374873548.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test[0:5]


287    219.0
211     70.0
72     202.0
321    230.0
73     111.0
Name: target, dtype: float64

In [46]:
y_pred_1[0:5]

array([[175.69168],
       [173.44817],
       [175.30165],
       [232.3956 ],
       [166.77913]], dtype=float32)

Hur kan mse vara 3600 om prediktionerna och testen ser ut som de gör ovan? Så mycket skilljer det inte. Root?
Efterssom skillnaden till mean kan vara både positivt och negativt borde vi ta roten, ser bättre ut.

In [47]:
# sqrt_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
# sqrt_2 = np.sqrt(mean_squared_error(y_test, y_pred_2))
# sqrt_3 = np.sqrt(mean_squared_error(y_test, y_pred_3))

Sista modellen bäst.

Bygg modell 4. Försök förbättra modell 3

Verkar som att batchnotmalization ger värre resultat.
Leakyrelu verkar ge lite bättre resultat


In [100]:
model_4 = Sequential()

model_4.add(Input(n_cols))
model_4.add(LeakyReLU())
model_4.add(Dense(500))
model_4.add(LeakyReLU())
model_4.add(Dense(500))
model_4.add(LeakyReLU())
model_4.add(Dense(500))
model_4.add(LeakyReLU())
model_4.add(Dense(500))
model_4.add(Dense(500, activation='relu'))
model_4.add(Dense(1, activation='linear'))


model_4.compile(optimizer='rmsprop', loss=keras.losses.mean_squared_error)

early_stop = EarlyStopping(patience=15)
model_4.fit(X_train, y_train, validation_split = 0.2, epochs = 50, callbacks=[early_stop])
y_pred_4 = model_4.predict(X_test)
res_model_4 = mean_squared_error(y_test, y_pred_4)

Epoch 1/50


c:\Users\olofs\anaconda3\envs\machine_learning\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


9/9 [==============================] - 2s 27ms/step - loss: 19225.5293 - val_loss: 3535.3733
Epoch 2/50
9/9 [==============================] - 0s 11ms/step - loss: 4325.2500 - val_loss: 3581.1990
Epoch 3/50
9/9 [==============================] - 0s 11ms/step - loss: 4186.0801 - val_loss: 3348.2844
Epoch 4/50
9/9 [==============================] - 0s 12ms/step - loss: 4621.6597 - val_loss: 3119.9609
Epoch 5/50
9/9 [==============================] - 0s 11ms/step - loss: 4158.0850 - val_loss: 3758.3345
Epoch 6/50
9/9 [==============================] - 0s 11ms/step - loss: 4087.1233 - val_loss: 3321.4995
Epoch 7/50
9/9 [==============================] - 0s 11ms/step - loss: 4382.0786 - val_loss: 3341.5454
Epoch 8/50
9/9 [==============================] - 0s 11ms/step - loss: 3805.4507 - val_loss: 4411.1030
Epoch 9/50
9/9 [==============================] - 0s 12ms/step - loss: 3032.6978 - val_loss: 5360.1934
Epoch 10/50
9/9 [==============================] - 0s 11ms/step - loss: 4231.7466 -

In [101]:
sqrt_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
sqrt_2 = np.sqrt(mean_squared_error(y_test, y_pred_2))
sqrt_3 = np.sqrt(mean_squared_error(y_test, y_pred_3))
sqrt_4 = np.sqrt(mean_squared_error(y_test, y_pred_4))

In [102]:
print(f"Result of model_1: {sqrt_1} \nResult of model_2: {sqrt_2} \nResult of model_3: {sqrt_3} \nResult of model_4: {sqrt_4}")

Result of model_1: 61.98317914094564 
Result of model_2: 147.12137049598817 
Result of model_3: 57.43972637098735 
Result of model_4: 55.03429193898584
